This notebook guides you through the code required to load and parse a text FAQ from a file named faq.txt from a local folder, split them into segments, and then create embeddings and ingest them into the vector database. You can use these embeddings for finding similarity in phrases that are similar in context or category. Embeddings are typically stored in a vector database which in this case Oracle Database 23ai.

In [1]:
# Read the source file and store chunks in an array
import os

def loadFAQs(directory_path):
   faqs = {}

   for filename in os.listdir(directory_path):
      if filename.endswith(".txt"):  # assuming FAQs are in .txt files
         file_path = os.path.join(directory_path, filename)

         with open(file_path) as f:
            raw_faq = f.read()

         filename_without_ext = os.path.splitext(filename)[0]  # remove .txt extension
         faqs[filename_without_ext] = [text.strip() for text in raw_faq.split('=====')]

   return faqs


The function will open all the .txt files in a specified folder which is txt-docs, read them, split the content using the ======== separator. It will then put all the resulting chunks in an array.
The array is stored inside a dictionary with the file name used as the key. This will be useful later if many other FAQ files are available inside the folder, helping to differentiate between the sources.

In [ ]:
# Store data from txt-docs
faqs = loadFAQs('./txt-docs')
faqs

We will call the loadFAQs function above with our data stored in txt-docs folder.

In [ ]:
docs = [{'text': filename + ' | ' + section, 'path': filename} for filename, sections in faqs.items() for section in sections]

# Sample the resulting data
docs[:2]

We will now prepare the source data to arrange the above dictionary in a way that is easy to ingest in the vector database.  This will store the data in the following format:
[{'text': 'faq | Who are you and what can you do?\n\nI am DORA, the Digital ORacle Assistant, a digital assistant working for Oracle EMEA. I can answer questions about Oracle Cloud (OCI) and especially about the Free Trial and Always Free programs.',
  'path': 'faq'}]
As you can see, for each chunk, we prepend the name of the source file to the text content, in our case, "faq |". This is a very simple way to ensure that some context is preserved with each chunk and vectorized later. It will help tremendously in the retrieval stage, when vector distances will be calculated between the question and each chunk.
Also, we state the name of the source file in the path component. It will be useful later if we want to display a link back to the source when using a chatbot.

In [ ]:
# Connect to the Oracle Database 23ai
un = "vector"
pw = "vector"
cs = "localhost/FREEPDB1"

import oracledb

connection = oracledb.connect(user=un, password=pw, dsn=cs)

oracledb.connect()function establishes a connection to an Oracle database using the provided credentials and connection details. The function takes username, password and dsn. The DSN (data source name) specifies the host, port and database service name to connect to.

In [ ]:
table_name = 'faqs'

with connection.cursor() as cursor:
    # Create the table
    create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id NUMBER PRIMARY KEY,
            payload CLOB CHECK (payload IS JSON),
            vector VECTOR
        )"""
    try:
        cursor.execute(create_table_sql)
    except oracledb.DatabaseError as e:
        raise

    connection.autocommit = True

We need a table inside our database to store our vectors and metadata.

In [ ]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-MiniLM-L12-v2')

We need an encoder to handle the vectorization for us. all-MiniLM-L12-v2 is a specific pre-trained model that is designed to be an encoder. It is based on the MiniLM (Mini Language Model) architecture, which is a lightweight version of transformer models like BERT.
Note:
•	Ignore the warning saying IProgress not found., among others.


In [ ]:
import array

# Define a list to store the data
data = [
   {"id": idx, "vector_source": row['text'], "payload": row} 
   for idx, row in enumerate(docs)
]

# Collect all texts for batch encoding
texts = [f"{row['vector_source']}" for row in data]

# Encode all texts in a batch
embeddings = encoder.encode(texts, batch_size=32, show_progress_bar=True)

# Assign the embeddings back to your data structure
for row, embedding in zip(data, embeddings):
   row['vector'] = array.array("f", embedding)

We go through all our chunks (stored in the docs dictionary) and encode the text content. Now we have a structure with all our chunks, including its context—the source file name, in this simple example—and the vector representation for each of them.
encoder.encode() method is used to generate embeddings (vector representations) for the input texts. It uses the encoder object, which is an instance of the SentenceTransformer class, loaded with a specific model (e.g., all-MiniLM-L12-v2).
batch_size=32 means that 32 sentences will be processed in parallel during each batch, which can be more efficient than processing each sentence individually.


In [ ]:
# Insert the chunks + vectors in the database
import json

with connection.cursor() as cursor:
    # Truncate the table
    cursor.execute(f"truncate table {table_name}")

    prepared_data = [(row['id'], json.dumps(row['payload']), row['vector']) for row in data]

    # Insert the data
    cursor.executemany(
        f"""INSERT INTO {table_name} (id, payload, vector)
        VALUES (:1, :2, :3)""",
        prepared_data
    )

    connection.commit()

Initially, we use a cursor object from the established database connection to execute a command that truncates the specified table. This operation ensures that all existing rows are removed, effectively resetting the table to an empty state and preparing it for fresh data insertion.
Subsequently, the code prepares a list of tuples containing the new data. Each tuple includes an id, a JSON-encoded payload, and a vector.
The json.dumps function is used to convert the payload into a JSON string format, ensuring that complex data structures are properly serialized for database storage.
We then utilize the cursor.executemany method to insert all prepared tuples into the table in a single batch operation. This method is highly efficient for handling bulk inserts, significantly reducing the number of database transactions and improving performance. Finally, the connection.commit method is called to commit the transaction, ensuring that all changes are saved and made permanent in the database.

In [ ]:
with connection.cursor() as cursor:
    # Define the query to select all rows from a table
    query = f"SELECT * FROM {table_name}"

    # Execute the query
    cursor.execute(query)

    # Fetch all rows
    rows = cursor.fetchall()

    # Print the rows
    for row in rows[:5]:
        print(row)

Above will examine what is stored in the table in our database and print first 5 rows. You should be able to see embeddings for the respective text chunks.